In [2]:
import pulp as p
import numpy as np
import math
import pandas as pd

from scipy import stats

import random

import math 

In [2]:
class weber_problem():
    
    def __init__(self, index, weight, x_coor, y_coor):
        self.index = np.array(index)
        self.weight = np.array(weight)
        self.x_coor = np.array(x_coor)
        self.y_coor = np.array(y_coor)
        
    def totalweight(self):
        return sum(self.weight)
    
    def weighteddist(self, x,y):
        return sum([self.weight[i]*math.sqrt((x-self.x_coor[i])**2+(y-self.y_coor[i])**2) for i in range(len(self.index))])
        
    def find_cog(self):
        x_cog = sum([self.weight[i]*self.x_coor[i]/sum(self.weight) for i in range(len(self.index))])
        y_cog = sum([self.weight[i]*self.y_coor[i]/sum(self.weight) for i in range(len(self.index))])
        cost_cog = self.weighteddist(x_cog, y_cog)/sum(self.weight)
        print('Center of Gravity x coordinate: ',x_cog)
        print('Center of Gravity y coordinate: ',y_cog)
        print('Center of Gravity avg. weighted dist: ',cost_cog)
        return x_cog, y_cog, cost_cog
    
    def weightedx(self): 
        return sum([self.weight[i]*self.x_coor[i]/sum(self.weight) for i in range(len(self.index))])
    
    def weightedy(self): 
        return sum([self.weight[i]*self.y_coor[i]/sum(self.weight) for i in range(len(self.index))])
    
    def find_weber(self):
        minX = self.x_coor.min()
        minY = self.y_coor.min()
        costmin = self.weighteddist(minX,minY)
    
        maxX = self.x_coor.max()
        maxY = self.y_coor.max()

        for i in np.arange(minX, maxX, 1):
            for j in np.arange(minY, maxY, 1):
                cost = self.weighteddist(i,j)
                if cost <= costmin:
                    x = i
                    y = j
                    costmin = cost
        costmin = costmin/sum(self.weight)
        
        print('weber x coordinate: ',x)
        print('weber y coordinate: ',y)
        print('weber avg. weighted dist: ',costmin)
        return x,y,costmin

In [17]:
#QQ3

#Using the same model for SandyCo, suppose the demand for each region is changed to 80 tons.

#What would the outcome be when you optimize this model?

x1a = p.LpVariable('1_to_a', lowBound = 0, cat = 'Integer')
x1b = p.LpVariable('1_to_b', lowBound = 0, cat = 'Integer')
x1c = p.LpVariable('1_to_c', lowBound = 0, cat = 'Integer')
x2a = p.LpVariable('2_to_a', lowBound = 0, cat = 'Integer')
x2b = p.LpVariable('2_to_b', lowBound = 0, cat = 'Integer')
x2c = p.LpVariable('2_to_c', lowBound = 0, cat = 'Integer')

sand = p.LpProblem('Sand Transportation', p.LpMinimize)

sand += 250*x1a + 325*x1b + 445*x1c + 275*x2a + 260*x2b + 460*x2c
sand += x1a + x1b + x1c <=100
sand += x2a + x2b + x2c <=125
sand += x1a + x2a >=80
sand += x1b + x2b >=80
sand += x1c + x2c >=80

sand.solve()

print('Optimal Solution: ', p.value(sand.objective))
print("Solution Status:", p.LpStatus[sand.status])

for item in sand.variables():
    print(item.name,' = ', item.varValue)


Optimal Solution:  77300.0
Solution Status: Infeasible
1_to_a  =  80.0
1_to_b  =  0.0
1_to_c  =  20.0
2_to_a  =  0.0
2_to_b  =  80.0
2_to_c  =  60.0


<b>Practice Problem 2: SteelCo</b>

SteelCo is a small steel distributor in Northern USA. They have two distribution centers (DCs), one in Allentown, PA, and one in Toledo, OH.

SteelCo has just received a large order from a construction contractor and will be supplying steel to four construction projects in the Northeast: one in Buffalo, one in Albany, one in Philadelphia, and one in Cleveland.

The contractor has ordered the following amounts to be shipped each week:

Buffalo: 40,500 pounds

Albany: 22,230 pounds

Philadelphia: 85,200 pounds

Cleveland: 47,500 pounds

Both DCs can each handle an additional 100,000 pounds of steel per week.

The transport costs for a pound of steel for the different DC-construction site combinations are given in the table below.

<img src = "w1pp1.png">

Suppose SteelCo wants to meet the customer's demand for steel at minimum cost. How many pounds of steel should be shipped from the DC in Allentown, PA, to the four construction projects, in total?

In [4]:
xaB = p.LpVariable('allentown_to_buffalo', lowBound = 0, cat = 'Integer')
xaA = p.LpVariable('allentown_to_albany', lowBound = 0, cat = 'Integer')
xaP = p.LpVariable('allentown_to_philadelphia', lowBound = 0, cat = 'Integer')
xaC = p.LpVariable('allentown_to_cleveland', lowBound = 0, cat = 'Integer')
xtB = p.LpVariable('toledo_to_buffalo', lowBound = 0, cat = 'Integer')
xtA = p.LpVariable('toledo_to_albany', lowBound = 0, cat = 'Integer')
xtP = p.LpVariable('toledo_to_philadelphia', lowBound = 0, cat = 'Integer')
xtC = p.LpVariable('toledo_to_cleveland', lowBound = 0, cat = 'Integer')

prob = p.LpProblem('Steel_Transportation', p.LpMinimize)

prob += 52*xaB + 32*xaA + 11*xaP + 69*xaC + 45*xtB + 84*xtA + 76*xtP + 15*xtC
prob += xaB + xtB >= 40500
prob += xaA + xtA >= 22230
prob += xaP + xtP >= 85200
prob += xaC + xtC >= 47500
prob += xaB + xaA + xaP + xaC <= 100000
prob += xtB + xtA + xtP + xtC <= 100000

prob.solve()

print('Optimal Solution: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ', item.varValue)


Optimal Solution:  4569920.0
Solution Status: Optimal
allentown_to_albany  =  14800.0
allentown_to_buffalo  =  0.0
allentown_to_cleveland  =  0.0
allentown_to_philadelphia  =  85200.0
toledo_to_albany  =  7430.0
toledo_to_buffalo  =  40500.0
toledo_to_cleveland  =  47500.0
toledo_to_philadelphia  =  0.0


<b>Practice Problem 3: Ralph Calvin Hilfiger (RCH) Industries</b>

Clothing manufacturer and retailer Ralph Calvin Hilfiger (RCH) Industries operates five (5) factories. They distribute their products through five (5) regional distribution centers (DCs) each of which supports a specific market. 

To reduce the number of direct shipments between factories and regional DCs, as well as improve the load factor of shipments, RCH has utilized a crossdocking (XD) facility for some time.  Owing to the success of their existing crossdocking facility (XD1), RCH has now decided to open an additional facility (XD2).

The clothes are transported through the network in small plastic totes via truck. The transportation costs for a tote between the five factories and the two crossdocking facilities are given in the table below.

<img src="w1pp31.png">

The transportation costs for a tote between the crossdocking facilities and the regional DCs are given in the table below.

<img src="w1pp32.png">

The five factories have the following production capacities per week (number of totes):

Factory 1: 200

Factory 2: 300

Factory 3: 100

Factory 4: 150

Factory 5: 220

The five regional DCs face the following demand per week (number of totes):

DC 1: 150

DC 2: 100

DC 3: 110

DC 4: 200

DC 5: 180

The crossdocking facilities have much spare capacity.

[Hint:  You need to build a transshipment model in your spreadsheet.  To check your spreadsheet, try setting each decision variable (the flow on each of the 20 arcs) = 1.  Your total cost should be $712 with 5 units going through each crossdock.  If this is not the case, check your model!]

Suppose RCH wishes to plan the flow of product from Factories to Crossdocks to Distribution Centers in order to minimize total costs.

How much should be shipped through the new facility, crossdocking facility two?


In [20]:
factories = ['Factory1', 'Factory2', 'Factory3', 'Factory4', 'Factory5']
factoriescap = [200,300,100,150,220]
dcs = ['DC1', 'DC2', 'DC3', 'DC4', 'DC5']
dcscap = [150,100,110,200,180]

crossdocks = ['Crossdock1','Crossdock2']
                   #crossdock1 #crossdock2
cost_fac_cross = [[30,          50], #factory1
                  [23,          66], #factory2
                  [35,          14], #factory3
                  [70,          12], #factory4
                  [65,          70]] #factory5

                #DC1 DC2 DC3 DC4 DC5
cost_cross_dc = [[12,25,22,40,41], #crossdock1
                 [65,22,23,12,15]] #crossdock2

factoriessupply = dict(zip(factories,factoriescap))
dcsdemand = dict(zip(dcs,dcscap))

fac_cross_dict = p.makeDict([factories, crossdocks], cost_fac_cross)
cross_dc_dict = p.makeDict([crossdocks, dcs], cost_cross_dc)

clothes = p.LpProblem('Clothes', p.LpMinimize)

fac_to_cross_route = [(f,c) for f in factories for c in crossdocks]

fac_to_cross_vars = p.LpVariable.dicts("route", (factories,crossdocks), lowBound = 0, cat = 'Integer')

cross_to_dc_route = [(c,d) for c in crossdocks for d in dcs]

cross_to_dc_vars = p.LpVariable.dicts("route", (crossdocks,dcs), lowBound = 0, cat = 'Integer')

clothes += p.lpSum([fac_to_cross_vars[f][c]*fac_cross_dict[f][c] for (f,c) in fac_to_cross_route]) \
            + p.lpSum([cross_to_dc_vars[c][d]*cross_dc_dict[c][d] for (c,d) in cross_to_dc_route])
                   
for f in factories:
    clothes += p.lpSum([fac_to_cross_vars[f][c] for c in crossdocks]) <= factoriessupply[f]
    
for d in dcs:
    clothes += p.lpSum([cross_to_dc_vars[c][d] for c in crossdocks]) >= dcsdemand[d]

for c in crossdocks:
    clothes += p.lpSum([fac_to_cross_vars[f][c] for f in factories]) == p.lpSum([cross_to_dc_vars[c][d] for d in dcs])
    
clothes.solve()

print('minimum cost: ', p.value(clothes.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in clothes.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  30220.0
Solution Status: Optimal
route_Crossdock1_DC1  =  150.0
route_Crossdock1_DC2  =  100.0
route_Crossdock1_DC3  =  110.0
route_Crossdock1_DC4  =  0.0
route_Crossdock1_DC5  =  0.0
route_Crossdock2_DC1  =  0.0
route_Crossdock2_DC2  =  0.0
route_Crossdock2_DC3  =  0.0
route_Crossdock2_DC4  =  200.0
route_Crossdock2_DC5  =  180.0
route_Factory1_Crossdock1  =  60.0
route_Factory1_Crossdock2  =  130.0
route_Factory2_Crossdock1  =  300.0
route_Factory2_Crossdock2  =  0.0
route_Factory3_Crossdock1  =  0.0
route_Factory3_Crossdock2  =  100.0
route_Factory4_Crossdock1  =  0.0
route_Factory4_Crossdock2  =  150.0
route_Factory5_Crossdock1  =  0.0
route_Factory5_Crossdock2  =  0.0


<b>Practice Problem 4: SandyCo - Multiple Optima</b>
    
For this problem, we will use the Transhipment Model for SandyCo (SandyCo Part 2). To recap, SandyCo has two (2) facilities that mine, clean, and sort sand for use in cement, children’s playboxes, and small beaches. They distribute the sand from their two (2) plants, through two (2) packaging centers (DCs), to three (3) different customer regions where it is packaged and sold.

Each plant has a specific maximum weekly supply of available sand and each region has an expected minimum weekly required demand. The cost to distribute a ton of sand differs between each plant and region pairing due to distance and other factors.

The maximum weekly supply from Plant 1 is 100 tons, from Plant 2 it is 125 tons. The expected minimum weekly required demand (in tons) is:

Region 1: 25
    
Region 2: 95
    
Region 3: 80
    
The transportation costs in dollars per ton of sand are given in the table below.

<img src = "w1pp41.png">

In the lesson we learned that in the optimal solution to the problem, the total cost is $69,200 per week. The solution presented in the lecture had the following weekly flows:

Plant 1 to DC B: 75

Plant 2 to DC A: 25

Plant 2 to DC B: 100

DC A to Region 1: 25

DC B to Region 2: 95

DC B to Region 3: 80

Input the flows above as decision-variables in your spreadsheet model. Don't run the solver. The spreadsheet will tell you that the total weekly cost is indeed $69,200 per week.

You hear from a colleague using another solver that there is another good solution to the problem. He tells you to consider following weekly flows:

Plant 1 to DC B: 75

Plant 2 to DC A: 105

Plant 2 to DC B: 20

DC A to Region 1: 25

DC A to Region 3: 80

DC B to Region 2: 95

Input the suggested flows instead of the flows from the lesson. What is the minimum weekly cost using the suggested flow volumes?

Round your answer to the nearest integer.

In [7]:
75*210 + 105*185 + 20*205 + 25*175 + 80*165 + 95*130

69200

Part2

After discussions with a number of carriers, SandyCo realizes that transportation costs to DC A need to be updated. The new costs are $180 per ton 

from Plant 1 to DC A and $190 per ton from Plant 2 to DC A.

How many tons of sand should be delivered from Plant 2 to DC B with the new costs?

Part3

After hearing that a rival sand distributor is facing economic difficulties, SandyCo wants to reconsider its demand estimates. Were the rival to default, SandyCo believes that their demand would increase by roughly 10% across the board, leading to the following minimum expected demands: Region 1 - 28, Region 2 - 105, and Region 3 - 87 tons respectively.

How many tons of sand should be delivered from DC A to Region 3 if the expected minimum demand was to increase by the amount specified? Note that the costs from Part 2 still apply.


In [2]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = [[180,       210], #sup1
                  [190,       205]] #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = [[175, 180, 165], #int1
                [235, 130, 145]] #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  68325.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  25.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  75.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  95.0
route_Intermediate2_Demand3  =  5.0
route_Supply1_Intermediate1  =  100.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  100.0


Part 4

The rival firm did not default - so your demand is back to the original values (Region 1: 25, Region 2: 95, and Region 3: 80).

The management of Plant 2 has suggested they may use some of the available space in the plant for an on-site packaging machine. With such a machine in place, Plant 2 would be able to bypass the DCs and ship directly to Region 1. Direct transport from Plant 2 to Region 1 costs $275 per ton. 

All other costs remain the same as in Part 2.

If Plant 2 invests in a on-site sand packaging machine, how many tons of sand should be delivered directly from Plant 2 to Region 1?


In [22]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = [[180,       210], #sup1
                  [190,       205]] #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = [[175, 180, 165], #int1
                [235, 130, 145]] #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup2todem1 = p.LpVariable('supply2_to_demand1', lowBound = 0, cat = 'Integer') #direct shipment from Plant2 (Supply2) to Region1(Demand1)

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + 275*sup2todem1 #added transportation cost from Plant2 to Region1
                   
for s in supply:
    if s == "Supply2": #add amount of sand transported from Plant2 to Region1 for Supply2
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + sup2todem1 <= supplyconstraint[s]
    else:
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]

for d in demand:
    if d == "Demand1": #add amount of sand transported from Plant2 to Region1 for Demand1
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + sup2todem1 >= demandconstraint[d]
    else:
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  66300.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  80.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  95.0
route_Intermediate2_Demand3  =  0.0
route_Supply1_Intermediate1  =  80.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  95.0
supply2_to_demand1  =  25.0


Part 5

After investing in the new machine, a breakdown in DC B reduces the weekly capacity of DC B to 80 tons. All other costs and demand remain the same as in Part 2. (Note that the bypass option from Part 4 remains.)

How many tons of sand should Plant 1 produce if the capacity of DC B drops to 80 tons? There is no capacity constraint for DC A.

In [24]:
supply = ['Supply1', 'Supply2']
supplycap = [100, 125]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [25,95,80]

intermediate = ['Intermediate1','Intermediate2']
intermediatecap = [10000000, 80] #capacity of DCB (Intermediate B) is 80 tons, while for DCA (Intermediate A), \
                                 #it is considered unlimited, hence the big number used

                   #int1      #int2
cost_sup_int   = [[180,       210], #sup1
                  [190,       205]] #sup2
                  
                #Dem1 Dem2 Dem3
cost_int_dem = [[175, 180, 165], #int1
                [235, 130, 145]] #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))
intermediateconstraint = dict(zip(intermediate,intermediatecap)) #added the constraint for the DCs (Intermediate)

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Sand', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

sup2todem1 = p.LpVariable('supply2_to_demand1', lowBound = 0, cat = 'Integer') #direct shipment from Plant2 (Supply2) to Region1(Demand1)

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + 275*sup2todem1 #added transportation cost from Plant2 to Region1
                   
for s in supply:
    if s == "Supply2": #add amount of sand transported from Plant2 to Region1 for Supply2
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) + sup2todem1 <= supplyconstraint[s]
    else:
        prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]

for d in demand:
    if d == "Demand1": #add amount of sand transported from Plant2 to Region1 for Demand1
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) + sup2todem1 >= demandconstraint[d]
    else:
        prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
for i in intermediate: #constraint for DCs (Intermediate) capacity
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) <= intermediateconstraint[i]
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  66675.0
Solution Status: Optimal
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  15.0
route_Intermediate1_Demand3  =  80.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  80.0
route_Intermediate2_Demand3  =  0.0
route_Supply1_Intermediate1  =  95.0
route_Supply1_Intermediate2  =  0.0
route_Supply2_Intermediate1  =  0.0
route_Supply2_Intermediate2  =  80.0
supply2_to_demand1  =  25.0


<b>Practice Problem 5: Food Delivery Startup</b>
    
You and a friend just started a food delivery startup. The idea is to deliver freshly made, organic, lunch-boxes to be sold in university cafeterias. You have secured deals with cafeterias at three universities and are now looking for a good location for the business.

To avoid spending too much time in traffic, you think it may be a good idea to find a location close to all three universities.

The different parameters (weights and coordinates for the locations) are summed up in the following table.

<img src="w1pp51.png">

What is the weighted X-coordinate?

What is the weighted Y-coordinate?

What is the average weighted distance from the business location to the 3 universities using the center of gravity technique?

What is the average weighted distance from the business location to the 3 universities using the Weber technique?

In [216]:
food = weber_problem(['MIT','HAR','YALE'],[250,150,100],[20,20,100],[30,40,150])

food.find_cog()
food.find_weber()


#def weighteddist(x,y, locations):
    #dist = 0
    #for i in range(len(locations)):
     #   dist = dist + locations.wgt.iloc[i]*math.sqrt((x-locations.x.iloc[i])**2+(y-locations.y.iloc[i])**2)
    #return dist
    #return sum([locations.wgt.iloc[i]*math.sqrt((x-locations.x.iloc[i])**2+\
     #                                           (y-locations.y.iloc[i])**2) for i in range(len(locations))])


Center of Gravity x coordinate:  36.0
Center of Gravity y coordinate:  57.0
Center of Gravity avg. weighted dist:  45.274676417067106
weber x coordinate:  20
weber y coordinate:  30
weber avg. weighted dist:  31.844410203711917


(20, 30, 31.844410203711917)

<b>Practice Problem 6: Temporary Networks</b>

When a natural disaster strikes, normal supply chains are disrupted and many vital supplies cannot reach those affected through the normal routes. To handle the first couple of days after the disaster, many specialized Non Governmental Organizations (NGOs) work hard to secure vital supplies such as food, blankets, and medicines, and deliver these to people in the affect areas. To do so, they set up temporary supply networks, which only operate as long as needed.

Suppose you are working with one such NGO to set up a temporary network to distribute disaster-kits in the aftermath of a hurricane. You have secured kits from your central supply facility that will be flown in regularly. The kits need to be delivered once per week as long as needed to eight (8) temporary shelters, which are located a few hours drive from the airport.

To simplify the operations, you aim to set up a logistics and distribution center (DC) that controls all distribution. All incoming shipments will be transported from the airport directly to the center by the military, and from the DC you will plan how the kits are delivered to the shelters.

You are choosing between five (5) locations for the DC. The distances between the five (5) potential locations and the eight (8) shelters are shown in the table below.

<img src="w1pp61.png">

The weekly demand at each shelter has been estimated as follows:

<img src="w1pp62.png">

The capacity of the DC will be limited to 150,000 kits per week. It costs $100,000 per week to set up and run a DC.

[Hint:  You need to build a facility location model in your spreadsheet, following the steps of the lessons.  To check your spreadsheet, try setting each decision variable (the flow on each of the 40 arcs and the five binary open-or-not variables) = 1.  Your total cost should be 500,199.  If this is not the case, check your model!]

Part1

Trucks that can operate on the roads after the hurricane are in short supply, so you want to choose the location of the DC to minimize the transport work (tonkm, or the total kit-miles travelled) in the temporary network. Note that transport work per kit is proportional to distance.

Which of the five locations should the NGO choose to minimize the transport work in the temporary network?

In [131]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = [[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]] #sup5
                  

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of kits travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_open[s]*100000 for s in supply]) #fixed cost of opening DC
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1 #number of DC maximum must be open
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d] #constraint must be equal or more than shelter demand

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 10000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 580910.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  0.0
open_Supply3  =  0.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  0.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  0.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4  =  0.0
route_S

Part 2

Due to road conditions, the shelters furthest away from the center may suffer from late and unreliable deliveries. As a result you are thinking about opening more than one DC.

To trade-off the fixed costs with the variable costs, you estimate that each transported mile costs $2 per kit.

Assuming the military is ok with delivering your supplies to more than one DC, what is the optimal number of DCs?

Part 3

You realize that the actual cost of setting up and operating a DC is not perfectly known. What is the optimal number of DCs if the fixed costs are instead $300,000 per week per DC? Assume the same transportation costs as in Part 2.

In [154]:
#Part 3

supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = [[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]] #sup5
                  

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d]*2 for (s,d) in sup_to_dem_route]) \ #2$ per tonkm
        + p.lpSum([sup_open[s]*300000 for s in supply])
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 10000000*sup_open[s] <= 0
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 1165660.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  0.0
open_Supply3  =  1.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply3_Demand1  =  14000.0
route_Supply3_Demand2  =  8000.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  22000.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  23000.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4

Part4

When you see the optimal solution from Part 3, you realize that under that solution, much of the demand is more than 2 miles away from a DC. How does the optimal solution change if we require that at least 60% of demand should be less than 2 miles from a DC?

In [73]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5']
supplycap = [150000, 150000, 150000, 150000, 150000 ]
demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8']
demandcap = [14000,8000,25000,22000,20000,17000,18500,23000]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[0.61, 3.32, 9.5, 6.52, 7.77, 1.92, 8.52, 9.75], #sup1
                  [6.04, 0.51, 1.34, 6.06, 0.22, 6.33, 4.61, 3.28], #sup2
                  [4.99, 2.41, 2.33, 3.95, 8.84, 7.94, 7.87, 0.94], #sup3
                  [5.58, 8.8, 6.32, 8.54, 5.15, 6.06, 9.42, 2.16], #sup4
                  [7.87, 9.64, 0.7, 5.92, 2.7, 0.26, 0.5, 3.4]]) #sup5

dist_less_than_2 = cost_sup_dem <= 2 #remember to use less than equal to sign

dist_less_than_2 = dist_less_than_2.astype(int) / sum(demandcap) #create matrix whose value is 1 if dist less than 2, 0 otherwise

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

dist_less_than_2_dict = p.makeDict([supply, demand], dist_less_than_2)

prob = p.LpProblem('Kits', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary')

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d]*2 for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_open[s]*300000 for s in supply])
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1

prob += p.lpSum([sup_to_dem_vars[s][d]*dist_less_than_2_dict[s][d] for (s,d) in sup_to_dem_route]) >= 0.6 #constraint 
                                                #that the % of demand within less than 2 miles is equal or more than 60%
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]

for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #change to == equal sign as to enforce hard
                                    #constraint (i.e.: otherwise the solution will just add kits to DCs just to increase 
                                    #the percentage of demand served less than 2 miles to be 60%

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000*sup_open[s] <= 0
    
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 1259780.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  1.0
open_Supply3  =  0.0
open_Supply4  =  0.0
open_Supply5  =  1.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply2_Demand1  =  14000.0
route_Supply2_Demand2  =  8000.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  0.0
route_Supply2_Demand5  =  20000.0
route_Supply2_Demand6  =  0.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  23000.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  0.0
route_Supply3_Demand3  =  0.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  0.0
route_Supply3_Demand6  =  0.0
route_Supply3_Demand7  =  0.0
route_Supply3_Demand8  =  0.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  0.0
route_Supply4_Demand4

<h3>Graded Assignment 1:  Locky Locke Inc.</h3>

You have finally been promoted to Senior Manager for the nuts and bolts category at Locky Locke Inc., a manufacturer with a large demand for bolts at its three (3) factories. You have surveyed the market and found four (4) suppliers for a certain steady selling SKU, ZA27R46.  Your factories' weekly demand for bolts are:

Factory 1: 9,500 cases/week

Factory 2: 6,500 cases/week

Factory 3: 17,000 cases/week

Each of the four suppliers can provide the ZA27R46, but they differ in unit price, weekly capacity, and distance to Locky Locke Inc. three factories. The unit prices and the weekly capacity for each of the four suppliers are shown below: 

Supplier 1:  U.S. Ainbolt  - Unit cost:  2.50 $/case  Capacity  9,500 cases/week

Supplier 2:  Der Bolt, Thun  - Unit cost:  3.20  $/case  Capacity  10,000 cases/week

Supplier 3: Li Tningbolt -  Unit cost:  1.90  $/case  Capacity  7,000 cases/week

Supplier 4:  M. Bolton -   Unit cost:  2.80  $/case  Capacity  9,000 cases/week

You are trying to plan the weekly flow of bolts from these suppliers to your factories. Based on the current contract with your transportation carrier, the unit transportation cost for a case of bolts is $0.05 per mile 

from any of the suppliers to any of the factories.  The distances between the four suppliers and the three factories are given in the table below.

<img src="w1ga1.png">

Part 1

Currently, the factories operate independently. Disregarding any of the other factories, which supplier should Factory 2 select in order to have the lowest total (purchase and transport) cost for them?

Part 2

You want to take a holistic view of bolt sourcing instead of having each factory select its supplier on its own. You decide to create a model to help you select the optimal sourcing assignment for all of the factories for the ZA27R46 bolts. Based on this optimization model, what is the total weekly cost for supplying bolts for Locky Locke Inc.? Be sure to include the purchase price as well as the transportation costs.

Part 3

Based on your optimization model, which supplier should Factory 2 select in order to have the lowest total (purchase and transport) cost for Locky Locke Inc.?

Part 4

You want to discuss the situation with the general manager of Factory 2. How much is the difference between Factory 2's total landed cost per week in Part 1 compared to the total landed cost per week from Part 3?

In [77]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4']
supplycap = np.array([9500, 10000, 7000, 9000])
supplyprice = np.array([2.5,3.2,1.9,2.8])

demand = ['Demand1', 'Demand2', 'Demand3',]
demandcap = np.array([9500, 6500, 17000])

                   #dem1 #dem2 #dem3
dist_sup_dem   = np.array([[120, 380, 806], #sup1
                  [465, 717, 853], #sup2
                  [553, 572, 830], #sup3
                  [915, 406, 564]]) #sup4

cost_sup_dem = dist_sup_dem*0.05
                  
#======================================================

print('PART1')

for i in range(len(supplycap)):
    transportcost = demandcap[1]*cost_sup_dem[i][1]
    purchasecost = demandcap[1]*supplyprice[i]
    totalcost = transportcost+purchasecost
    print("cost factory"+str(i+1)+": ",totalcost)
    
#======================================================
    

supplyconstraint = dict(zip(supply,supplycap))
supplypricedict = dict(zip(supply,supplyprice))

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Bolts', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route]) \
        + p.lpSum([sup_to_dem_vars[s][d]*supplypricedict[s] for (s,d) in sup_to_dem_route]) #transport cost + purchase cost as obj. function

                  
#prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
#prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]
    
prob.solve()

print('\nPART2')

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

#===================================================================

print("\nPART4")

TLC1 = 139750
TLC2 = 198250
print('Difference in cost for Factory2 in Part1 vs Part2: ',TLC1-TLC2)



PART1
cost factory1:  139750.0
cost factory2:  253825.0
cost factory3:  198250.0
cost factory4:  150150.0

PART2
minimum cost: 923575.0
Solution Status: Optimal
route_Supply1_Demand1  =  9500.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  7500.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  6500.0
route_Supply3_Demand3  =  500.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  9000.0

PART4
Difference in cost for Factory2 in Part1 vs Part2:  -58500


Part 5

You decide to re-negotiate with Supplier 1 (U.S. Ainbolt) to increase their capacity. They agree to increase it to 14,000 cases per week. How many cases per week does Supplier 1 (U.S. Ainbolt) send to Factory 2 in the new optimal solution?
Input your answer in integers.

Based on this optimization model, what is the total weekly cost for supplying bolts for Locky Locke Inc.? Be sure to include the purchase price as well as the transportation costs.

In [80]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4']
supplycap = np.array([14000, 10000, 7000, 9000])
supplyprice = np.array([2.5,3.2,1.9,2.8])

demand = ['Demand1', 'Demand2', 'Demand3',]
demandcap = np.array([9500, 6500, 17000])

                   #dem1 #dem2 #dem3
dist_sup_dem   = np.array([[120, 380, 806], #sup1
                  [465, 717, 853], #sup2
                  [553, 572, 830], #sup3
                  [915, 406, 564]]) #sup4

cost_sup_dem = dist_sup_dem*0.05
            
supplyconstraint = dict(zip(supply,supplycap))
supplypricedict = dict(zip(supply,supplyprice))

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Bolts', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous')
                  
prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_to_dem_vars[s][d]*supplypricedict[s] for (s,d) in sup_to_dem_route])

                  
#prob += p.lpSum([sup_open[s] for s in supply]) <= 5
                  
#prob += p.lpSum([sup_open[s] for s in supply]) >= 1
                   
for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s]
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) >= demandconstraint[d]
    
prob.solve()

print('\nPART5')

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


PART5
minimum cost: 872050.0
Solution Status: Optimal
route_Supply1_Demand1  =  9500.0
route_Supply1_Demand2  =  4500.0
route_Supply1_Demand3  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  3000.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand2  =  2000.0
route_Supply3_Demand3  =  5000.0
route_Supply4_Demand1  =  0.0
route_Supply4_Demand2  =  0.0
route_Supply4_Demand3  =  9000.0


<h3>Graded Assignment 2: GoClean&Co</h3>

GoClean&Co (GCC) hired you as its Logistics Manager in charge of transshipments. GoClean&Co is a joint public-private energy company dedicated to generating clean and safe nuclear energy. GCC uses a new technology based on radioactive pellets. These pellets can generate electricity for years in GCC's nuclear plants. Once the radioactive pellets are depleted, they are removed from active use and collected at the plants for removal at the end of the month. Once a month, the depleted pellets are removed from the plants, transported to an inspection site for examination, and then moved again to a dedicated storage site for safekeeping.

You have been informed that two of GCC's nuclear plants, called Plant One and Plant Two, need to move out depleted pellets in the following amounts:

Plant One wants to ship out up to 280 pellets per month

Plant Two wants to ship out up to 360 pellets per month

This is how many pellets these two plants want to ship out per month. However, the exact number of pellets that will actually be moved out from each plant depends on the space that is available at the dedicated storage sites that would ultimately receive them. GCC is mandated by law to move as many pellets out of the plants as the storage sites are able to receive. You have been informed that two of GCC's dedicated storage sites, called Storage X and Storage Y, are ready to receive depleted pellets from GCC's plants, in the following amounts:

Storage X is ready to receive a total of 305 pellets per month

Storage Y is ready to receive a total of 325 pellets per month

Since the two plants want to ship out more pellets per month than the two storage sites can receive, you know that some pellets will have to stay in their plant of origin until a later time. This is not a problem, however, since every one of GCC's plants has the capacity to temporarily store whatever pellets are not shipped out, and the law allows this.

At this time, only one inspection site is available to receive the pellets from the two plants, inspect them, and send them out to the two storage sites. It is called Inspection Site A. Notice that inspection sites do not have storage capacity: the pellets come in, are inspected, and continue their trip to the storage sites.

Moving radioactive material is not easy and therefore it is not cheap either. There is a lot of planning and security involved. Each shipment of pellets has to be closely monitored and protected to prevent the loss of radioactive material. Also, the vehicles transporting the material have to move relatively slowly, to avoid accidents. Therefore, it is very expensive to transport the radioactive pellets from the plant to the inspection site, and from there to the storage site. Based on estimates and historical data, GCC has calculated the following transportation costs, in dollars per pellet moved:

It costs 13.58 dollars to move one pellet from Plant One to Inspection Site A

It costs 16.54 dollars to move one pellet from Plant Two to Inspection Site A

It costs 7.57 dollars to move one pellet from Inspection Site A to Storage X

It costs 16.46 dollars to move one pellet from Inspection Site A to Storage Y

You decide to optimize the transhipment of depleted pellets from the plants to the storage sites (stopping on the way at Inspection Site A, of course) in order to minimize the overall monthly transportation cost. In this optimized solution, how many pellets will Plant Two be able to ship out to the Inspection Site A every month? (This question is not graded).

In [82]:

q1X = p.LpVariable('plant1_to_storageX', lowBound = 0, cat = 'Integer')
q1Y = p.LpVariable('plant1_to_storageY', lowBound = 0, cat = 'Integer')
q2X = p.LpVariable('plant2_to_storageX', lowBound = 0, cat = 'Integer')
q2Y = p.LpVariable('plant2_to_storageY', lowBound = 0, cat = 'Integer')

prob = p.LpProblem('Radioactive_Transportation', p.LpMinimize)

prob += (13.58+7.57)*q1X + (13.58+16.46)*q1Y + (16.54+7.57)*q2X + (16.54+16.46)*q2Y
prob += q1X + q2X == 305
prob += q1Y + q2Y == 325
prob += q1X + q1Y <= 280
prob += q2X + q2Y <= 360

prob.solve()

print('Optimal Solution: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ', item.varValue)

Optimal Solution:  17249.75
Solution Status: Optimal
plant1_to_storageX  =  0.0
plant1_to_storageY  =  280.0
plant2_to_storageX  =  305.0
plant2_to_storageY  =  45.0


Part 2

GCC has calculated the following transportation costs for Inspection Site B, in dollars per pellet moved:

It costs 12.6 dollars to move one pellet from Plant One to Inspection Site B

It costs 16.2 dollars to move one pellet from Plant Two to Inspection Site B

It costs 9.45 dollars to move one pellet from Inspection Site B to Storage X

It costs 15.64 dollars to move one pellet from Inspection Site B to Storage Y

As before, you will optimize the transhipment of depleted pellets from the plants to the storage sites, stopping first at the inspection sites, so as to minimize the overall monthly transportation cost. How much will this cost GCC every month? .

In [85]:
supply = ['Supply1', 'Supply2']
supplycap = [280, 360]
demand = ['Demand1', 'Demand2']
demandcap = [305, 325]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = [[13.58,       12.6], #sup1
                  [16.54,       16.2]] #sup2
                  
                #Dem1 Dem2
cost_int_dem = [[7.57, 16.46], #int1
                [9.45, 15.64]] #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  16693.550000000003
Solution Status: Optimal
route_Intermediate1_Demand1  =  305.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  325.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  305.0
route_Supply2_Intermediate2  =  45.0


New Plant and Storage Site

You have just received two e-mails from your boss. The first informs you that a third nuclear plant, called Plant Three, is ready to move out up to 340 depleted pellets per month. The second tells you that a third dedicated storage site, called Storage Z, is ready to receive a total of 320 depleted pellets per month from GCC's plants. GCC has calculated the following transportation costs for the new plant and storage site, in dollars per pellet moved:

It costs 16.37 dollars to move one pellet from Plant Three to Inspection Site A

It costs 10.81 dollars to move one pellet from Plant Three to Inspection Site B

It costs 15.69 dollars to move one pellet from Inspection Site A to Storage Z

It costs 15.41 dollars to move one pellet from Inspection Site B to Storage Z

Again you will optimize the transhipment of depleted pellets from the three plants to the three storage sites, stopping first at the inspection site, so as to minimize the overall monthly transportation cost. In this optimal solution, how many pellets is Plant One sending to Inspection Site B?

In this optimized solution, how much is the overall monthly transportation cost for GCC? (This question is graded.) Hint: The answer is more than 21000 but less than 26000.

In [86]:
supply = ['Supply1', 'Supply2', 'Supply3']
supplycap = [280, 360, 340]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [305, 325, 320]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = [[13.58,       12.6], #sup1
                  [16.54,       16.2], #sup2
                  [16.37,       10.81]] #sup3
                  
                #Dem1 #Dem2   #Dem3
cost_int_dem = [[7.57, 16.46, 15.69], #int1
                [9.45, 15.64, 15.41]] #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24976.15
Solution Status: Optimal
route_Intermediate1_Demand1  =  305.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  325.0
route_Intermediate2_Demand3  =  320.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  305.0
route_Supply2_Intermediate2  =  25.0
route_Supply3_Intermediate1  =  0.0
route_Supply3_Intermediate2  =  340.0


Part 4

GCC has learned that a new regulation, to be enacted soon, will require that no depleted pellets be stored at the plants past the monthly removal date. To acomodate this, the storage sites have increased the total number of pellets they are willing to receive every month, as follows:

Storage X is now ready to receive a total of 315 pellets per month

Storage Y is now ready to receive a total of 335 pellets per month

Storage Z is now ready to receive a total of 330 pellets per month

The new regulations also make it easier for pellets to be moved, which reduces all the transportation costs in the network by 3.13 percent. (This means you have to multiply all the previous transportation costs by 0.9687 in order to get the new costs.) One last time, you will optimize the transhipment of depleted pellets from the three plants to the three storage sites, stopping first at the inspection sites, so as to minimize the overall monthly transportation cost. (Hint: In this optimized solution, Plant Two is shipping 315 pellets per month to Inspection Site A.)

In this optimized solution, how much is the overall monthly transportation cost for GCC?

In [88]:
supply = ['Supply1', 'Supply2', 'Supply3']
supplycap = [280, 360, 340]
demand = ['Demand1', 'Demand2', 'Demand3']
demandcap = [315, 335, 330]

intermediate = ['Intermediate1','Intermediate2']
                   #int1      #int2
cost_sup_int   = np.array([[13.58,       12.6], #sup1
                  [16.54,       16.2], #sup2
                  [16.37,       10.81]]) #sup3
                  
                #Dem1 #Dem2   #Dem3
cost_int_dem = np.array([[7.57, 16.46, 15.69], #int1
                [9.45, 15.64, 15.41]]) #int2

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.9687)
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*0.9687)

prob = p.LpProblem('Radioactive_Waste', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route]) \
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s]
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d]

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand])
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  25042.590225
Solution Status: Optimal
route_Intermediate1_Demand1  =  315.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  335.0
route_Intermediate2_Demand3  =  330.0
route_Supply1_Intermediate1  =  0.0
route_Supply1_Intermediate2  =  280.0
route_Supply2_Intermediate1  =  315.0
route_Supply2_Intermediate2  =  45.0
route_Supply3_Intermediate1  =  0.0
route_Supply3_Intermediate2  =  340.0


<h3>Graded Assignment 3: Notebooks for the world</h3>
    
Since you are attending the SC2x course, your best friend Sarah knows that you learned how to calculate, for a given group of cities with their Cartesian coordinates, a location for a distribution center that would minimize the distance. Thus, she asks you to calculate the coordinates for a distribution center, so that she can minimize its distance to ten cities in Mexico."

You found a map of Mexico online, and after asking Sarah for a list of the ten major cities in Mexico, you decide to use an improvised grid to assign horizontal and vertical coordinates to the cities. These coordinates, shown in the table below, use as origin a point in the top left corner of the map. You tried to make these units as close to kilometers as possible. However, you prefer to think of them as relative coordinates on an arbitrary Euclidean grid.

<img src="w1ga31.png">

"Do you have weights for these cities?", you ask your friend. 

"No", she says, "but let's assume they all have equal weight, a weight of 1 for example".

Part 0

Giving each city equal weight, calculate the location for the distribution center that would minimize the distance to the ten cities provided. Solve this as a continuous, distance-minimizing, location (or Weber) problem with equal weights for all cities.

Which of the following coordinates is nearest to the location found? This question is not graded, it just serves to help you check whether you are on the right track!



In [208]:
mexico = weber_problem(['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                      [1,1,1,1,1,1,1,1,1,1],[110,159,120,160,148,145,179,170,153,203],[120,80,81,42,92,42,26,40,61,35])

mexico.find_weber()

weber x coordinate:  153
weber y coordinate:  61
weber avg. weighted dist:  33.07588744975341


(153, 61, 33.07588744975341)

Part 2

Curious about Sarah's request, you ask her what she is working on. She shares with you that together with her co-workers she is volunteering for an organization that will soon distribute notebooks to every school-aged child throughout Mexico. You suggest that solving this problem with equal weights is not good enough, since some cities surely have more schoolchildren than others. A quick online search provides you with approximate values for the populations of the different areas where the cities are located. Using these population values, you propose to your friend using weight values, as listed in the table below.

City	Weights

Chihuahua	926926

Ciudad Victoria	322322

Durango	519519

Mexico City	20439419

Monterrey	4524520

Morelia	785785

Oaxaca	650650

Puebla	2931929

San Luis Potosi	1042041

Villahermosa	640640

Using these weights, solve this as a continuous, demand-weighted distance-minimizing, location (or Weber) problem.

What is the horizontal coordinate?

What is the vertical coordinate?
 

In [209]:
locations = pd.DataFrame(index=['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                         data={'wgt':[926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640],\
                               'x':[110,159,120,160,148,145,179,170,153,203],\
                               'y':[120,80,81,42,92,42,26,40,61,35]})


mexico = weber_problem(['Chihuahua','Ciudad Victoria','Durango','Mexico City',\
                                'Monterrey','Morelia','Oaxaca','Puebla','San Luis Potosi','Villahermosa'],\
                      [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640],[110,159,120,160,148,145,179,170,153,203],[120,80,81,42,92,42,26,40,61,35])

mexico.find_weber()

weber x coordinate:  160
weber y coordinate:  42
weber avg. weighted dist:  14.234610918746556


(160, 42, 14.234610918746556)

Part 3

You decide to estimate the distance of distributing the notebooks from a DC in one of a set of selected cities to the school children in another city. Given that you only need a quick and dirty estimate, you assume an Euclidean space and a circuity factor of 1.25, which gives you the distances in the table below (where cities are identified using the first three letters in their name). Regardless of which city the DC is located in, there will also be transportation from the DC to the school children within that same city. For distances inside each city, you assume 5 distance units, except for Mexico City, for which you use 10 distance units since it is a larger city.

The distances between the cities are given in your arbitrary grid units, which may or may not be kilometers.

<img src="w1ga32.png">

After thinking about it for a while, your friend tells you that they may want to open two DCs. She wants these DCs to be located in two of the five candidate cities (Chihuahua, Mexico City, Monterrey, Puebla, San Luis Potosi), but they should not be located in the same city. The plan would be to distribute all the hundreds of thousands of notebooks through these two DCs, using trucks. Let's assume that the cost of opening and operating such a DC in any of these cities would be the about the same, and that the cost of outbound transportation from the DC to the cities is about the same on a per unit per mile basis.

In which two cities would you recommend that they open these two DCs, in order to minimize the outbound transportation costs? Select the answer options that apply. Use the same demand weights as in Part 2. (Hint: If you are puzzled by the fact that we are asking for this without providing any costs for DCs or trucks, you should start playing with different values and see how the model behaves.)

In [145]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5'] #no cap on supply

demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8', 'Demand9', 'Demand10']

demandcap = [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = [[5,719,503,1158,590,1069,1458,1250,913, 1575], #sup1
                  [1158,475,698,10,643,188,310,127,253,545], #sup2
                  [590,203,376,643,5,626,911,706,393,990], #sup3
                  [1250,519,808,127,706,313,208,5,338,417], #sup4
                  [913,249,482,253,393,258,545,338,5,704]] #sup5
                  

demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem)

prob = p.LpProblem('Books', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of items travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_open[s]*10000000 for s in supply]) #fixed cost of opening DC
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 2 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 2389242876.0
Solution Status: Optimal
open_Supply1  =  0.0
open_Supply2  =  1.0
open_Supply3  =  1.0
open_Supply4  =  0.0
open_Supply5  =  0.0
route_Supply1_Demand1  =  0.0
route_Supply1_Demand10  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply1_Demand9  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand10  =  640640.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  20439419.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  785785.0
route_Supply2_Demand7  =  650650.0
route_Supply2_Demand8  =  2931929.0
route_Supply2_Demand9  =  1042041.0
route_Supply3_Demand1  =  926926.0
route_Supply3_Demand10  =  0.0
route_Supply3_Demand2  =  322322.0
route_Supply3_Demand3  =  519519.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  4524520.0
route

Part 4

After some market research on carriers in Mexico, you calculate an average transportation cost of $1 per distance unit per 1001 notebooks 

(e.g. you would pay $100

to move 1001 notebooks 100 distance units). You also estimate that the fixed cost of operating each distribution center in one of these cities is $464,000 dollars per year. 

The yearly demands in the cities are approximated by the weights used in the previous parts.

Assuming that these values are correct and applicable throughout the country, how many distribution centers would you recommend opening?

Input your answer in integers.

Where would you recommend that they open these DCs, in order to minimize the total cost (fixed + transportation)?

In [147]:
supply = ['Supply1', 'Supply2', 'Supply3', 'Supply4' , 'Supply5'] #no cap on supply

demand = ['Demand1', 'Demand2', 'Demand3', 'Demand4', 'Demand5', 'Demand6', 'Demand7', 'Demand8', 'Demand9', 'Demand10']

demandcap = [926926,322322,519519,20439419,4524520,785785,650650,2931929,1042041,640640]

                   #dem1 #dem2 #dem3 #dem4 #dem5 #dem6 #dem7 #dem8
cost_sup_dem   = np.array([[5,719,503,1158,590,1069,1458,1250,913, 1575], #sup1
                  [1158,475,698,10,643,188,310,127,253,545], #sup2
                  [590,203,376,643,5,626,911,706,393,990], #sup3
                  [1250,519,808,127,706,313,208,5,338,417], #sup4
                  [913,249,482,253,393,258,545,338,5,704]]) #sup5


demandconstraint = dict(zip(demand,demandcap))

sup_dem_dict = p.makeDict([supply, demand], cost_sup_dem*1/1001) #transport cost of 1dollar per distance unit per 1001 books

prob = p.LpProblem('Books', p.LpMinimize)

sup_to_dem_route = [(s,d) for s in supply for d in demand]

sup_to_dem_vars = p.LpVariable.dicts("route", (supply,demand), lowBound = 0, cat = 'Continuous') #amount of items travelling between DC and shelter
                  
sup_open = p.LpVariable.dicts("open", supply, cat = 'Binary') #binary variables DC open or close

prob += p.lpSum([sup_to_dem_vars[s][d]*sup_dem_dict[s][d] for (s,d) in sup_to_dem_route])\
        + p.lpSum([sup_open[s]*464000 for s in supply]) #fixed cost of opening DC of 464000 dollars each
                  
prob += p.lpSum([sup_open[s] for s in supply]) <= 5 #number of DC minimum must be open
                  
prob += p.lpSum([sup_open[s] for s in supply]) >= 1 #number of DC maximum must be open
                   
#for s in supply:
#    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) <= supplyconstraint[s] #constraint must be less than DC capacity
                  
for d in demand:
    prob += p.lpSum([sup_to_dem_vars[s][d] for s in supply]) == demandconstraint[d] #constraint must be equal to cities demand
    #using '<=' will result in a higher optimal cost compared to using '==' because in this problem there is no constraint on the supply

for s in supply:
    prob += p.lpSum([sup_to_dem_vars[s][d] for d in demand]) - 100000000000000000*sup_open[s] <= 0 #linking constraint if any channel from DC is flowing, the DC must be open
    
prob.solve()

print('minimum cost:', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost: 3175608.0
Solution Status: Optimal
open_Supply1  =  1.0
open_Supply2  =  1.0
open_Supply3  =  1.0
open_Supply4  =  1.0
open_Supply5  =  0.0
route_Supply1_Demand1  =  926926.0
route_Supply1_Demand10  =  0.0
route_Supply1_Demand2  =  0.0
route_Supply1_Demand3  =  0.0
route_Supply1_Demand4  =  0.0
route_Supply1_Demand5  =  0.0
route_Supply1_Demand6  =  0.0
route_Supply1_Demand7  =  0.0
route_Supply1_Demand8  =  0.0
route_Supply1_Demand9  =  0.0
route_Supply2_Demand1  =  0.0
route_Supply2_Demand10  =  0.0
route_Supply2_Demand2  =  0.0
route_Supply2_Demand3  =  0.0
route_Supply2_Demand4  =  20439419.0
route_Supply2_Demand5  =  0.0
route_Supply2_Demand6  =  785785.0
route_Supply2_Demand7  =  0.0
route_Supply2_Demand8  =  0.0
route_Supply2_Demand9  =  1042041.0
route_Supply3_Demand1  =  0.0
route_Supply3_Demand10  =  0.0
route_Supply3_Demand2  =  322322.0
route_Supply3_Demand3  =  519519.0
route_Supply3_Demand4  =  0.0
route_Supply3_Demand5  =  4524520.0
route_Supply3_Demand6  =

<h1>Week 2</h1>

<h3>Practice Problem 2: Papper</h3>

Papper is a paper goods manufacturer. One of their main paper products is a paper towel, which is produced in two (2) plants in the contiguous U.S. From the plants, the paper towels are sent to nine (9) regional warehouses (RW), from where they are then shipped to customers across the U.S.

The distribution manager at Papper wants to reduce the number of direct transports in the network, reduce the inventory levels at the plants, and move inventory closer to the regional warehouses, by introducing a central warehouse (CW) where all finished paper towels are stored before further shipment. That is, the idea is to have the towels flowing from the plants to the CW and then to the RWs. The CW will have a small safety stock, but everything that comes into the CW during a week should also leave the CW during that week.

The distribution manager has found five (5) candidate locations for the CW that he is now choosing between. Capacity is not an issue at any of them.

All data, including weekly demand data (in number of boxes), is found here (Excel) or here (Libre Office). Assume transportation cost is $1 per mile per box of paper towels.

<img src="w2pp21.png">

Out of the candidate locations, which central warehouse location minimizes total costs (while meeting the constraints)?

In [15]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 1 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  349490.0
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  140.0
route_Intermediate2_Demand2  =  180.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  175.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  320.0
route_Intermediate2_Demand8  =  280.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_

Part 2

Suppose the distribution manager wants at least 60% of regional demand to be within 70 miles of the central warehouse(s).

How many central warehouses are used in the optimal solution, if we introduce the constraint that 60% of regional demand must be within 70 miles of a central warehouse?


In [60]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

dist_within_limit = cost_int_dem <= 70 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint, changed to hard constraint in this case

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC max must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimal must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.6 #60% of demand must be within less than 70 miles of CW
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  490882.0
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  134.0
route_Intermediate2_Demand8  =  0.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  140.0
route_Intermediate3_Demand2  =  180.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Inte

Part 3

Suppose the same service constraints as in Part 2 applies.

After investigating the transport market, the manager realizes that he has completely overestimated the inbound transportation costs. In fact, inbound transportation costs can be reduced to $0.3 per mile and box.

How does this change the optimal network design (compared to Part 2)?

In [74]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1500, 500])
supply_variable_cost = np.array([8,13])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8', 'Demand9']
demandcap = np.array([140, 180, 240,210,175,130,320,280,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100000,90000,80000,70000,60000])
intermediate_variable_cost = np.array([8,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8 #Dem9
cost_int_dem = np.array([[89,78,69,95,85,73,88,73,86], #int1
                         [90,91,63,61,94,58,89,90,55], #int2
                         [81,82,71,98,62,71,85,75,72], #int3
                         [73,97,55,75,86,54,62,99,59], #int4
                        [85,98,93,82,87,58,98,72,50]]) #int5

dist_within_limit = cost_int_dem <= 70 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*0.3) #cost for inbound transport changed to 0.3 
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Paper_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 100000000000000*int_open[i] <= 0 #linking constraint between flow and CW open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC max must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimal must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.6
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  437072.5
Solution Status: Optimal
open_Intermediate1  =  0.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  0.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate1_Demand9  =  0.0
route_Intermediate2_Demand1  =  140.0
route_Intermediate2_Demand2  =  180.0
route_Intermediate2_Demand3  =  240.0
route_Intermediate2_Demand4  =  210.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  0.0
route_Intermediate2_Demand9  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Interm

<h3>Practice Problem 3: Cocup</h3>
    
Cocup is a paper cup manufacturer making generic coffee cups used for take away coffee around the world. They have one (1) manufacturing plant and two (2) Distribution Centers (DCs), as well as eight (8) regional sales offices (SOs) with warehouses on site. Paper cups are made at the plant and shipped by external carriers through the DCs to the sales offices.

The weekly demands at the eight SOs are shown below.

<img src ="w2pp31.png">

Last week the Director of Marketing at Cocup signed a contract with a growing coffee retail chain, making Cocup the retailer's sole supplier of coffee cups. The retailer will be supplied through sales offices SO1, SO2 and SO3, increasing the demand at these offices to:

SO1 = 121 thousand cups

SO2 = 96 thousand cups

SO3 = 166 thousand cups

Since the manufacturing plant is operating close to its capacity of 1,250 thousand cups per week, Cocup has started to investigate whether they should use a contract manufacturer, now that the demand for their products is increasing. They have found one contract manufacturer that meets their requirements. This contractor can provide the cups at 50dollars per thousand cups and has the capacity to provide 200 thousand cups per week. This is more expensive than the 10dollars per thousand cups of producing in-house, but because the contractor is located closer to the two DCs than Cocup's plant is, it will lead to more production capacity closer to the DCs.

Cocup has also started to investigate whether they should rent space from a third party's distribution center to support the sales offices. They are considering three (3) more DCs. The fixed and variables costs of these Potential DCs are found in the table below together with the fixed and variable costs of the current DCs (DC 1 and DC 2).

<img src = "w2pp32.png">

The distances from the DCs to the sales offices are

<img src = "w2pp33.png">

and the distances from the plants to the DCs are

<img src = "w2pp34.png">

There are no capacity limits at the DCs at this point. Transportation costs are 1$ per thousand cups and mile, for both inbound and outbound transportation.

The logistics manager of Cocup has asked you to build an optimization model to support their decision-making. The objective of the model is to minimize costs.

Based on the results of your model, would you recommend the logistics manager to use the contract manufacturer with the current distribution network when demand increases? Note that the current network has two (2) DCs, and eight (8) SOs.

In [49]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,800000000,700000000,600000000]) #set DC3 - 5's fixed cost to be very high which is the same as saying only DC1 and DC2 will be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 2
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  178671.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  121.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  166.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  180.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Int

Part 2

The manager is uncertain about how much demand will actually increase at the three sales offices. Consequently, the manager is interested in understanding how sensitive your conclusions are to the assumptions about the increase in demand.

Suppose demand increases by 20% at each of the three sales offices instead, so that the demands are SO1: 132, SO2: 96, and SO3: 186. Under this scenario, using your model, would you recommend the logistics manager to use the contract manufacturer with the current distribution network?

In [51]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([132,96,186,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,800000000,700000000,600000000]) #set DC3 - 5's fixed cost to be very high which is the same as saying only DC1 and DC2 will be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost))

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #cost of 1 dollar per mile per box

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 2 #number of DC minimum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 2
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  183595.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  0.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  132.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  186.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  180.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  0.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Int

Part 3

After receiving additional information from Marketing, the logistics manager is fairly confident that the original demand estimates from (Part 1) are correct (SO1: 121, SO2: 96, and SO3: 166 thousand cups). For the remainder of the analyses, she wants you to use these values.

The logistics manager is also interested in exploring adding more DCs to the network, if it reduces total costs. If Cocup does not use the contractor, what is the optimal number of DCs?

Part 4

When you show your analysis to the manager she comments that the solution seems to be largely driven by the transportation costs. She tells you that based on the contracts they have with their carriers, there is much uncertainty about the real transportation costs. In fact, fluctuations in oil prices have lead to very volatile prices over the last years and she anticipates this may continue.

Since transportation costs are historically low at this point in time, she wants you to investigate what will happen if costs were to increase in the close future. She therefore asks you to redo the analysis from Part 3 with higher transportation costs.

Consider the same situation as in Part 3. If outbound transportation costs increase by 50%, which of the following statements is/are correct?

In [72]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1.5) #outbound transport cost increased by 50%

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  199962.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  0.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  130.0
route_Intermedi

Part 5

The manager had another comment on you analysis in Part 3. Suppose outbound transportation costs remain at 1dollar per 1,000 cups and mile. Even with several DCs, this leads to a large portion of demand at the SOs being more than 50 miles from a DC.

Consider the same situation as in Part 3. If 75% of demand should be within 50 miles of a DC, what implications does it have on the volumes at DC 2 compared to a solution where there is no such constraint?

In [77]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

dist_within_limit = cost_int_dem <= 50 #remember to use less than or equal to sign

dist_within_limit = dist_within_limit.astype(int) / sum(demandcap)

dist_within_limit_dict = p.makeDict([intermediate, demand], dist_within_limit)


supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) #outbound transport cost increased by 50%

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 1000000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 5 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += p.lpSum([int_to_dem_vars[i][d]*dist_within_limit_dict[i][d] for (i,d) in int_to_dem_route]) >= 0.75 # 75% of demand must be within limit
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)

    
#Explanation

#DC 2 is cheap to operate in scale but is relatively far away from several of the SOs. 
#Increasing the LOS means that we should use one more DC (DC 5) and reduce the volume at DC 2. 
#This is once again a trade-off between service and cost.

#To see this, include the LOS-constraint and put it on a low level (say 50%). We see that the flow 
#through DC2 is 586 thousand cups. Then increase the constraint to 75%. The flow through DC2 is 
#now reduced to 465 thousand cups.



minimum cost:  169913.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  1.0
open_Intermediate5  =  1.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  0.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  0.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Intermedi

Part 6

The last comment the manager had about your analysis in Part 3 has to do with prioritization. Even though it may be optimal to use several third party DCs, Cocup may want to start with one to ensure that all processes are working before ramping up.

Consider the same situation as in Part 3. If Cocup was to use only one of the third party DCs, which one should it use?

In [82]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])

intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) >= demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += int_open['Intermediate1'] == 1 #force constraint Intermediate1 must be open

prob += int_open['Intermediate2'] == 1 #force constraint Intermediate2 must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  167415.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  96.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  210.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  121.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  175.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  130.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  160.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  166.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Inter

<h3>Practice Problem 4: Uncertainty at Cocup</h3>

Consider again Cocup, the paper cup manufacturer. Cocup has one (1) manufacturing plant and two (2) Distribution Centers (DCs), as well as eight (8) regional sales offices (SOs) with warehouses on site. Paper cups are made at the plant and shipped by external carriers through the DCs to the sales offices.

As you may remember, the manufacturing plant is operating close to its capacity of 1,250 thousand cups per week. Because of this, Cocup has started to investigate if they should use a contract manufacturer. They have found one contract manufacturer that met their requirements. This contractor can provide the cups at 50dollar per thousand cups and has the capacity to provide 200 thousand cups per week. This is more expensive than the 10dollar per thousand cups of producing in-house, but because the contractor is located closer to the two DCs than Cocup's plant is, it will lead to more production capacity closer to the DCs.

Cocup has also started to investigate whether they should rent space from a third party's distribution center to support the sales offices. They are considering three (3) more DCs, to add to the two (2) DCs they already have in their network. Transportation costs are 1$ per thousand cups and mile, for both inbound and outbound transportation.

As you may remember, the logistics manager of Cocup was skeptical about the demand estimates. The report from Marketing calmed him down momentarily, but still he cannot let go of the idea of the demand estimates being off.

He asks you to perform a few more analyses, based on the model you created last week.

To address the uncertainty in demand, he wants you to assume demands at the Sales Regions are normally distributed with a CV of 0.1. In particular, he wants you to run a number of scenarios based on realizations from these normal distributions.

Run your model over 10 generated demand realizations, where demands are assumed to follow normal distributions with a CV (Coef. of Variation; CV = stddev/mean) of 0.1.

Based on your FIRST RUN, should Cocup use the contractor?

In [3]:
supply = ['Supply1', 'Supply2']
supplycap = np.array([1250, 250])
supply_variable_cost = np.array([10,50])

demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5', 'Demand6','Demand7', 'Demand8']
demandcap = np.array([121,96,166,175,210,130,180,160])
demandcap = [int(stats.norm(demandcap[i], 0.1*demandcap[i]).ppf(random.random())) for i in range(len(demandcap))] 
#add randomness to the demand which is modeled as a normal distribution with a CV of 0.1


intermediate = ['Intermediate1','Intermediate2','Intermediate3','Intermediate4','Intermediate5']
intermediate_fixed_cost = np.array([100,100,80,70,60]) #all DC has potential to be used
intermediate_variable_cost = np.array([10,10,25,30,50])

                          #int1 #int2 #int3 #int4 #int5
cost_sup_int   = np.array([[77,   48,   41,   57,   94], #sup1
                           [54,    33,   21,   94,   44]]) #sup2
                  
                         #Dem1 #Dem2 #Dem3 #Dem4 #Dem5 #Dem6 #Dem7 #Dem8
cost_int_dem = np.array([[62,55,114,106,51,76,122,80], #int1
                         [63,92,90,50,88,47,136,47], #int2
                         [76,85,56,65,129,40,89,100], #int3
                         [42,128,74,70,45,56,81,131], #int4
                        [77,135,118,59,124,93,41,94]]) #int5

supplyconstraint = dict(zip(supply,supplycap))
demandconstraint = dict(zip(demand,demandcap))

supplyvariablecost = dict(zip(supply,supply_variable_cost))

intermediatevariablecost = dict(zip(intermediate,intermediate_variable_cost))
intermediatefixedcost = dict(zip(intermediate,intermediate_fixed_cost)) 

sup_int_dict = p.makeDict([supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per box
int_dem_dict = p.makeDict([intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Cup_Production', p.LpMinimize)

sup_to_int_route = [(s,i) for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (supply,intermediate), lowBound = 0, cat = 'Integer')

int_open = p.LpVariable.dicts("open", intermediate, cat = 'Binary')

int_to_dem_route = [(i,d) for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[s][i]*sup_int_dict[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[i][d]*int_dem_dict[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([supplyvariablecost[s]*sup_to_int_vars[s][i] for (s,i) in sup_to_int_route])\
            + p.lpSum([intermediatevariablecost[i]*int_to_dem_vars[i][d] for (i,d) in int_to_dem_route])\
            + p.lpSum([intermediatefixedcost[i]*int_open[i] for i in intermediate])
                   
for s in supply:
    prob += p.lpSum([sup_to_int_vars[s][i] for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
    
for d in demand:
    prob += p.lpSum([int_to_dem_vars[i][d] for i in intermediate]) == demandconstraint[d] #RW demand constraint

for i in intermediate:
    prob += p.lpSum([sup_to_int_vars[s][i] for s in supply]) == p.lpSum([int_to_dem_vars[i][d] for d in demand]) #conservation of flow constraint
    
for i in intermediate:
    prob += p.lpSum([int_to_dem_vars[i][d] for d in demand]) - 10000000000*int_open[i] <= 0 #linking constraint between flow and Intermediate open
    
prob += p.lpSum([int_open[i] for i in intermediate]) <= 3 #number of DC maximum must be open
                  
prob += p.lpSum([int_open[i] for i in intermediate]) >= 1 #number of DC minimum must be open

prob += int_open['Intermediate1'] == 1 #force constraint Intermediate1 must be open

prob += int_open['Intermediate2'] == 1 #force constraint Intermediate2 must be open
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  168325.0
Solution Status: Optimal
open_Intermediate1  =  1.0
open_Intermediate2  =  1.0
open_Intermediate3  =  1.0
open_Intermediate4  =  0.0
open_Intermediate5  =  0.0
route_Intermediate1_Demand1  =  0.0
route_Intermediate1_Demand2  =  86.0
route_Intermediate1_Demand3  =  0.0
route_Intermediate1_Demand4  =  0.0
route_Intermediate1_Demand5  =  201.0
route_Intermediate1_Demand6  =  0.0
route_Intermediate1_Demand7  =  0.0
route_Intermediate1_Demand8  =  0.0
route_Intermediate2_Demand1  =  138.0
route_Intermediate2_Demand2  =  0.0
route_Intermediate2_Demand3  =  0.0
route_Intermediate2_Demand4  =  176.0
route_Intermediate2_Demand5  =  0.0
route_Intermediate2_Demand6  =  132.0
route_Intermediate2_Demand7  =  0.0
route_Intermediate2_Demand8  =  162.0
route_Intermediate3_Demand1  =  0.0
route_Intermediate3_Demand2  =  0.0
route_Intermediate3_Demand3  =  177.0
route_Intermediate3_Demand4  =  0.0
route_Intermediate3_Demand5  =  0.0
route_Intermediate3_Demand6  =  0.0
route_Inter

<h3>Practice Problem 5: RCH Industries' Differentiation</h3>
    
Ralph Calvin Hilfiger (RCH) Industries - the clothing manufacturer from Week 1 - has found that different clothing styles have very different demands in the five (5) markets. For instance, the more stylish items sell relatively more in Europe, while the more leisurely items seem to do much better in the U.S. To accommodate this, they want to update the network design, and rethink the product mix produced in each of the five factories.

As before, RCH distributes their products from their five (5) factories through (2) cross docking facilities (XD) on to five (5) regional distribution centers (DCs), each of which supports a specific market. The clothes are transported through the network in small plastic totes via truck. There are no capacity constraints at the cross docks at this point.

RCH has grouped their clothing items into two major style categories: Stylish and Leisure. The demand for the categories in the five markets, as well as production capacities and transportation costs are found below

<img src="w2pp51.png">

Suppose RCH wishes to plan the flow of products from Factories to Crossdocks to Distribution Centers in order to minimize total costs, subject to the constraints. Build a spreadsheet model of the network.

In the optimal solution, how many factories are dedicated to producing only one style group?

In [16]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                        #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([[150,   300,  90,   140,  220],#type1
                      [200,   300,  70,    30,  220]])#type2


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([types, supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types: #note the posisitoning of the for loops and how it corresponds to the constraints we are trying to model
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) <= supplyconstraint[t][s] #plant cap constraint

    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #RW demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

prob += p.lpSum([sup_to_int_vars[t]['Supply1'][i] for t in types for i in intermediate]) <= 200 #constraint of total production in Sup1 must not exceed a certain number
prob += p.lpSum([sup_to_int_vars[t]['Supply2'][i] for t in types for i in intermediate]) <= 300 #constraint of total production in Sup2 must not exceed a certain number
    
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  23630.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  0.0
route_Type1_Intermediate1_Demand3  =  0.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  45.0
route_Type1_Intermediate2_Demand3  =  70.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  130.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  80.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  140.0
route_Type1_Supply5_Intermediate1  =  0.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Inte

Part 2

Suppose the production equipment is fully flexible, so that any factory can produce any mix of the two styles, as long as the following capacity constraints are not violated for the combined total production of both types: Factory 1 - 200; Factory 2 - 300; Factory 3 - 100; Factory 4 - 150, Factory 5 - 220 (note that these are the same capacity limits as in the Practice Problem in Week 1). Suppose RCH wants to minimize costs.

In the optimal solution, how many factories are now dedicated to producing only one style group?

In [28]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                      #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([200,   300,  100,   150,  220]) #constraint is changed to total production capacity of any product type


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types:
    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for s in supply: #note the for loops inside the lp.sum, compare with when the constraint is per type per factory
        prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for i in intermediate]) <= supplyconstraint[s] #plant cap constraint is changed to be capacity for both types combined         

prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24230.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  45.0
route_Type1_Intermediate1_Demand3  =  70.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  0.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  0.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  245.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  100.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  5.0
route_Type1_Supply5_Intermediate1  =  0.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Inter

Part 3

RCH does not feel comfortable with your solution from Part 2 - too few factories are producing both styles, they claim.

If at least 10 units of each style needs to be produced in every factory, how much will the optimal cost increase compared to the solution in Part 2? Answer in dollars without the dollar sign, e.g., if the solution is $45.45, write 45.

In [30]:
types = ['Type1', 'Type2']
supply = ['Supply1', 'Supply2','Supply3','Supply4','Supply5']
                      #sup1 #sup2 #sup3 #sup4 #sup5
supplycap = np.array([200,   300,  100,   150,  220]) #constraint is changed to total production capacity of any product type


demand = ['Demand1', 'Demand2', 'Demand3','Demand4', 'Demand5']
                      #dem1 #dem2 #dem3 #dem4 #dem5
demandcap = np.array([[130,  45,   70,   100,  5], #type1
                      [15,   45,   40,   100,175]])#type2

intermediate = ['Intermediate1','Intermediate2']

                            #sup1       #sup2       #sup3       #sup4      #sup5
                            #int1#int2 #int1#int2  #int1#int2  #int1#int2 #int1#int2
cost_sup_int   = np.array([[[30,   50],[23,    66],[35,    14],[70,   12],[65,   70]], #type1
                           [[33,   55],[25,    73],[39,    15],[77,   13],[12,   14]]]) #type2
                  
                            #int1                     #int2
                         #Dem1#Dem2#Dem3#Dem4#Dem5   #Dem1#Dem2#Dem3#Dem4#Dem5
cost_int_dem = np.array([[[12, 25,  22,  40,    41], [65,  22,  23,  12,   15]],#type1
                         [[13, 28,  24,  44,    45], [72,  24,  25,  13,   17]]]) #type2

supplyconstraint = p.makeDict([supply], supplycap)
demandconstraint = p.makeDict([types, demand], demandcap)

sup_int_dict = p.makeDict([types, supply, intermediate], cost_sup_int*1) #cost of 1 dollar per mile per unit
int_dem_dict = p.makeDict([types, intermediate, demand], cost_int_dem*1) 

prob = p.LpProblem('Clothes_Production', p.LpMinimize)

sup_to_int_route = [(t,s,i) for t in types for s in supply for i in intermediate]

sup_to_int_vars = p.LpVariable.dicts("route", (types, supply,intermediate), lowBound = 0, cat = 'Integer')

int_to_dem_route = [(t,i,d) for t in types for i in intermediate for d in demand]

int_to_dem_vars = p.LpVariable.dicts("route", (types, intermediate,demand), lowBound = 0, cat = 'Integer')

prob += p.lpSum([sup_to_int_vars[t][s][i]*sup_int_dict[t][s][i] for (t,s,i) in sup_to_int_route])\
            + p.lpSum([int_to_dem_vars[t][i][d]*int_dem_dict[t][i][d] for (t,i,d) in int_to_dem_route])

for t in types:
    for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for i in intermediate]) >= 10 #enforce the constraint of minimum number of product type must be produced each
    
    for d in demand:
        prob += p.lpSum([int_to_dem_vars[t][i][d] for i in intermediate]) >= demandconstraint[t][d] #demand constraint

    for i in intermediate:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for s in supply]) == p.lpSum([int_to_dem_vars[t][i][d] for d in demand]) #conservation of flow constraint

for s in supply:
        prob += p.lpSum([sup_to_int_vars[t][s][i] for t in types for i in intermediate]) <= supplyconstraint[s] #plant cap constraint
        
prob.solve()

print('minimum cost: ', p.value(prob.objective))
print("Solution Status:", p.LpStatus[prob.status])

for item in prob.variables():
    print(item.name,' = ',item.varValue)


minimum cost:  24930.0
Solution Status: Optimal
route_Type1_Intermediate1_Demand1  =  130.0
route_Type1_Intermediate1_Demand2  =  45.0
route_Type1_Intermediate1_Demand3  =  70.0
route_Type1_Intermediate1_Demand4  =  0.0
route_Type1_Intermediate1_Demand5  =  0.0
route_Type1_Intermediate2_Demand1  =  0.0
route_Type1_Intermediate2_Demand2  =  0.0
route_Type1_Intermediate2_Demand3  =  0.0
route_Type1_Intermediate2_Demand4  =  100.0
route_Type1_Intermediate2_Demand5  =  5.0
route_Type1_Supply1_Intermediate1  =  10.0
route_Type1_Supply1_Intermediate2  =  0.0
route_Type1_Supply2_Intermediate1  =  225.0
route_Type1_Supply2_Intermediate2  =  0.0
route_Type1_Supply3_Intermediate1  =  0.0
route_Type1_Supply3_Intermediate2  =  90.0
route_Type1_Supply4_Intermediate1  =  0.0
route_Type1_Supply4_Intermediate2  =  15.0
route_Type1_Supply5_Intermediate1  =  10.0
route_Type1_Supply5_Intermediate2  =  0.0
route_Type2_Intermediate1_Demand1  =  15.0
route_Type2_Intermediate1_Demand2  =  0.0
route_Type2_Int